### A2.4.3. Loop Vectorization

> *Loop vectorization transforms a scalar loop into one that processes multiple data elements per iteration using SIMD instructions, exploiting data-level parallelism.*

**Explanation:**

**Loop vectorization** replaces scalar operations on individual elements with vector operations on multiple elements simultaneously. A loop processing one `float` per iteration becomes one processing 4 or 8 floats per iteration using SIMD instructions (SSE, AVX, NEON).

**Requirements for Vectorization:**

- **No loop-carried dependencies** — each iteration must be independent of the others (or dependencies must be reducible).
- **Known trip count** (or a runtime check on the trip count).
- **No complex control flow** — or it must be convertible to predicated/masked operations.
- **Memory access pattern** — contiguous (stride-1) accesses are cheapest; gathers/scatters are more expensive.

**LLVM's Loop Vectorizer:**

- Chooses a **vectorization factor (VF)** — how many elements to process per vector iteration.
- Chooses an **interleave count (IC)** — how many vector iterations to unroll for ILP.
- Generates a **scalar epilogue** for remaining iterations when trip count is not a multiple of VF.
- Emits **remarks** (`-Rpass=loop-vectorize`) reporting vectorization decisions.

**Example:**

```c
// Scalar:
for (int i = 0; i < n; i++)
    c[i] = a[i] + b[i];

// Vectorized (VF=4):
for (int i = 0; i < n; i += 4)
    c[i:i+4] = a[i:i+4] + b[i:i+4];  // single SIMD add
```

In [ ]:
import numpy as np

size = 16
array_a = np.arange(size, dtype=np.float32)
array_b = np.arange(size, dtype=np.float32) * 2

scalar_result = np.empty(size, dtype=np.float32)
for index in range(size):
    scalar_result[index] = array_a[index] + array_b[index]

vectorized_result = array_a + array_b

vectorization_factor = 4
vector_iterations = size // vectorization_factor
remainder = size % vectorization_factor

print(f"Array size: {size}")
print(f"Vectorization factor: {vectorization_factor}")
print(f"Vector iterations: {vector_iterations}")
print(f"Scalar epilogue iterations: {remainder}")
print(f"Results match: {np.array_equal(scalar_result, vectorized_result)}")
print(f"Result: {vectorized_result}")

**References:**

[📘 LLVM Project. *Auto-Vectorization in LLVM.*](https://llvm.org/docs/Vectorizers.html)

[📘 Aho, A., Lam, M., Sethi, R. & Ullman, J. (2006). *Compilers: Principles, Techniques, and Tools (2nd ed.).* Addison-Wesley.](https://www.pearson.com/en-us/subject-catalog/p/compilers-principles-techniques-and-tools/P200000003155)

---

[⬅️ Previous: Dead Code Elimination](./02_dead_code_elimination.ipynb) | [Next: MLIR Dialects ➡️](../05_MLIR/01_mlir_dialects.ipynb)